In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import display
import ipywidgets as widgets

from backtest_runner import BacktestRunner
from strategy.ma_crossover_strategy import MovingAverageCrossoverStrategy
from strategy.rsi_strategy import RSIStrategy
from strategy.bb_strategy import BollingerBandsStrategy

# Sample historical data
# Replace this with real data from your loader
from utils import load_sample_data
historical_data = load_sample_data("NSE:INFY", interval="day", days=120)

# Mapping strategy name to class
strategy_map = {
    "MA Crossover": MovingAverageCrossoverStrategy,
    "RSI": RSIStrategy,
    "Bollinger Bands": BollingerBandsStrategy
}

# Widgets
strategy_dropdown = widgets.Dropdown(
    options=strategy_map.keys(),
    value="Bollinger Bands",
    description="Strategy:"
)

capital_slider = widgets.IntSlider(
    value=10000, min=1000, max=100000, step=500,
    description="Capital:"
)

run_button = widgets.Button(
    description="Run Backtest",
    button_style='success'
)

output = widgets.Output()

def plot_results(trades, df):
    fig, ax = plt.subplots(figsize=(12, 6))
    df["date"] = pd.to_datetime(df["date"])
    ax.plot(df["date"], df["close"], label="Close Price", color="black")

    # Mark buy/sell
    for trade in trades:
        t_date = pd.to_datetime(trade["date"])
        color = "green" if trade["action"] == "BUY" else "red"
        ax.scatter(t_date, trade["price"], color=color, marker="^" if color=="green" else "v", s=100)

    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%d'))
    plt.xticks(rotation=45)
    ax.set_title("Trade Signals")
    ax.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

def on_run_clicked(_):
    output.clear_output()

    strategy_name = strategy_dropdown.value
    strategy_class = strategy_map[strategy_name]
    capital = capital_slider.value

    runner = BacktestRunner(strategy_class, capital=capital)
    result = runner.run(historical_data)

    with output:
        print(f"🔎 Strategy: {strategy_name}")
        print(f"💼 Capital: ₹{capital}")
        print(f"📈 Total PnL: ₹{result['total_pnl']}")
        print(f"📊 Win Rate: {result['win_rate_percent']}%")
        print("🧾 Trades:")
        display(pd.DataFrame(result["trades"]))
        plot_results(result["trades"], pd.DataFrame(historical_data))

run_button.on_click(on_run_clicked)

# UI layout
ui = widgets.VBox([strategy_dropdown, capital_slider, run_button, output])
display(ui)
